In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
from nilearn.plotting import plot_design_matrix, plot_stat_map, view_img
from nilearn.reporting import get_clusters_table
from nilearn import datasets
import glob
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/03_level3')
from make_design_files import make_design_files

In [ ]:
def get_filt_tval_img(reg, mnum = "1", tstat="1",threshold=0.95,level3_path='model1'):
    
    level3_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/derivatives/nilearn/glm/level3/%s'%(level3_path)
    
    pos_pval_fn = '%s/rand_model%s_%s_tfce_corrp_tstat%s.nii.gz'%(reg, mnum, reg, tstat)
    neg_pval_fn = '%s/rand_model%s_%s_tfce_corrp_tstat%s.nii.gz'%(reg, mnum, 'neg_'+reg, tstat)
    tval_fn = '%s/rand_model%s_%s_tstat%s_tfce.nii.gz'%(reg, mnum, reg, tstat)
    
    pos_pval_img = os.path.join(level3_path, pos_pval_fn)
    neg_pval_img = os.path.join(level3_path, neg_pval_fn)
    tval_img = os.path.join(level3_path, tval_fn)
    
    pos_pval_img = nib.load(pos_pval_img)
    neg_pval_img = nib.load(neg_pval_img)
    tval_img = nib.load(tval_img)
    
    tval_data = tval_img.get_fdata()
    pos_pval_data = pos_pval_img.get_fdata()
    neg_pval_data = neg_pval_img.get_fdata()
    
    filt_tval_data = np.where(pos_pval_data > threshold, tval_data, np.where(neg_pval_data > threshold, tval_data, 0))
    filt_tval_img = nib.Nifti1Image(filt_tval_data.astype(np.float64), tval_img.affine)
    
    return filt_tval_img

def plot_filt_tval_img(reg, mnum = "1", tstat="1", threshold=0.95, interactive=False, level3_path='model1'):
    filt_tval_img = get_filt_tval_img(reg, mnum = mnum, tstat=tstat, threshold=threshold, level3_path=level3_path)
    if(len(np.unique(filt_tval_img.get_fdata())) == 1):
        print('Nothing survives correction for %s, model = %s, tstat = %s'%(reg, mnum, tstat))
    elif interactive:
        view = view_img(filt_tval_img, 
             draw_cross=False,
             title='%s'%(reg))
        
        return view
    else:
        print('Plotting tvalues filtered for corrected p values > 0.05')
        plot_stat_map(filt_tval_img, 
             draw_cross=False,
             title='%s'%(reg))

# Model 1: fractal + stim + choice + reward

In [ ]:
# Model 2: 